In [ ]:
import os
import json
from typing import List, Dict, Set
from google import genai
from google.genai.errors import APIError

# --- 1. CONFIGURATION GLOBALE ---

# Chemins (Doivent être corrects par rapport à votre structure de dossier)
FICHIER_VOCABULAIRE_CONCEPTS = "token_concept.json" 
FICHIER_THESAURUS_SORTIE = "Thesaurus_20_Concepts.json"

# Les 20 Termes Préférés (TP) ciblés pour le thésaurus
CIBLES_CONCEPTS: List[str] = [
    "Arrosage", "Luminosité", "Types de sol / substrat", "pH du sol", 
    "Température", "Humidité ambiante", "Engrais / nutrition NPK", "Rempotage", 
    "Plantation (mise en terre / pot)", "Drainage", "Propagation / reproduction", 
    "Taille et élagage", "Maladies fongiques", "Parasites courants", 
    "Toxicité (humains & animaux)", "Croissance et développement", 
    "Stress hydrique / thermique / lumineux", "Cycle de vie (annuelle / vivace / arbuste...)", 
    "Origine géographique & biomes", "Technologies horticoles modernes (LED, capteurs, hydroponie…)"
]

# --- 2. INITIALISATION ET CHARGEMENT DU VOCABULAIRE ---

try:
    # Le client lit la clé depuis la variable d'environnement GEMINI_API_KEY
    client = genai.Client() 
    print("✅ Initialisation réussie (Gemini Client).")
except Exception as e:
    print(f"🚨 ERREUR d'initialisation du client : {e}")
    exit()

def charger_vocabulaire_concepts(filepath: str) -> Set[str]:
    """Charge l'ensemble des tokens depuis le fichier JSON des concepts."""
    if not os.path.exists(filepath):
        print(f"🚨 ERREUR : Fichier vocabulaire non trouvé à {filepath}. Annulation.")
        return set()
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
            if isinstance(data, list):
                return set(data)
            else:
                print("🚨 ERREUR : Le fichier vocabulaire n'est pas une liste de mots.")
                return set()
    except Exception as e:
        print(f"❌ ERREUR lors du chargement du vocabulaire : {e}")
        return set()

# Chargement de tous les tokens du corpus concept
TERMES_CONCEPTS: Set[str] = charger_vocabulaire_concepts(FICHIER_VOCABULAIRE_CONCEPTS)

if not TERMES_CONCEPTS:
    print("\n❌ Arrêt : Vocabulaire de base vide ou introuvable.")
    exit()

print(f"✅ {len(TERMES_CONCEPTS)} tokens de concepts chargés pour le LLM.")


# --- 3. GÉNÉRATION PAR LLM ---

# Augmenter le nombre de tentatives en cas d'erreur API 503 temporaire
MAX_RETRIES = 3 

def generer_thesaurus_llm(tp_cibles: List[str], vocabulaire: Set[str]) -> str:
    """
    Construit le prompt unique avec les tokens et appelle l'API Gemini.
    """
    
    # 1. Sérialisation de la liste de vocabulaire pour l'inclusion dans le prompt
    vocabulaire_liste_str = json.dumps(sorted(list(vocabulaire)), ensure_ascii=False, indent=2)
    
    # 2. Construction du Prompt
    prompt_template = f"""
Role: Tu es un expert en conception de thésaurus (ISO 25964) pour la recherche botanique et agricole. Ton objectif est de générer des entrées de thésaurus complètes au format JSON pour les Termes Préférés (TP) suivants : {tp_cibles}.

Règles de Contenu Stricte (Basées sur le Vocabulaire Brut):
1. SÉLECTION EXCLUSIVE : Les valeurs pour les relations **TP, NT (Terme Plus Spécifique), BT (Terme Plus Général) et RT (Terme Associé)** DOIVENT ÊTRE CHOISIES **UNIQUEMENT** parmi les tokens exacts de la "Liste de Vocabulaire Brut" fournie ci-dessous. Ces tokens représentent les mots-clés indexés du corpus PDF.
2. ENRICHISSEMENT (UF) : La section **UF (Synonymes/Non-Descripteurs)** doit être enrichie avec des variations linguistiques courantes (expressions usuelles, anglicismes, ou mots en Darija si pertinents) que les utilisateurs pourraient taper. Ces termes sont les **SEULS** qui peuvent être **générés librement** par toi et ne pas être présents dans la "Liste de Vocabulaire Brut".
3. HIÉRARCHIE : Tu dois identifier pour chacun des {len(tp_cibles)} TP des termes plus spécifiques (NT) et/ou plus généraux (BT) en parcourant la liste de vocabulaire brut. Si un TP n'a pas de relation pertinente dans le vocabulaire, laisse le champ comme une liste vide `[]`.
4. NOTE D'APPLICATION (SN) : Fournir une brève Note d'Application (SN) décrivant le concept pour chaque TP.

Format de Sortie Strict:
Génère STRICTEMENT un seul objet JSON. Chaque clé de l'objet doit être un Terme Préféré (TP), et sa valeur doit être un objet contenant les champs BT, NT, RT, UF, et SN.

---
### DONNÉES D'ENTRÉE BRUTES DU CORPUS ({len(vocabulaire)} tokens)
---
Liste de Vocabulaire Brut (Tokens disponibles pour TP, NT, BT, RT):
{vocabulaire_liste_str}

--- SORTIE JSON ATTENDUE ---
"""
    
    print(f"\n[--- ENVOI DU PROMPT À GEMINI pour {len(tp_cibles)} TP ---]")
    
    for attempt in range(MAX_RETRIES):
        try:
            print(f"Tentative {attempt + 1}/{MAX_RETRIES}...")
            response = client.models.generate_content(
                model='gemini-2.5-flash',
                contents=prompt_template,
                config={"response_mime_type": "application/json"}
            )
            # Succès : Retourne la sortie JSON
            return response.text
            
        except APIError as e:
            if '503 UNAVAILABLE' in str(e) and attempt < MAX_RETRIES - 1:
                delay = 15  # Attendre 15 secondes avant de réessayer
                print(f"❌ Erreur 503 (Surchargé). Nouvelle tentative dans {delay} secondes...")
                time.sleep(delay)
            else:
                print(f"❌ ERREUR API critique : Échec après {attempt + 1} tentatives. Détail : {e}")
                return None
        except Exception as e:
            print(f"❌ ERREUR inattendue lors de l'appel LLM : {e}")
            return None
    
    return None # Si toutes les tentatives ont échoué

# --- 4. SCRIPT PRINCIPAL (Exécution) ---

if __name__ == "__main__":
    
    # 1. GÉNÉRATION DU THESAURUS PAR LLM
    thesaurus_json_str = generer_thesaurus_llm(CIBLES_CONCEPTS, TERMES_CONCEPTS)
    
    # 2. SAUVEGARDE ET AFFICHAGE
    if thesaurus_json_str:
        try:
            # Charger la chaîne JSON renvoyée par le LLM
            thesaurus_data = json.loads(thesaurus_json_str)
            
            # Sauvegarder dans le fichier de sortie
            with open(FICHIER_THESAURUS_SORTIE, 'w', encoding='utf-8') as f:
                json.dump(thesaurus_data, f, ensure_ascii=False, indent=2)
                
            print(f"\n✅ Thésaurus généré avec succès et enregistré dans '{FICHIER_THESAURUS_SORTIE}'.")
            
            # Affichage d'un aperçu
            print("\n--- APERÇU DU RÉSULTAT FINAL (Premières entrées) ---")
            
            # Afficher seulement quelques entrées pour ne pas inonder la console
            apercu_data = {}
            for i, (tp, data) in enumerate(thesaurus_data.items()):
                apercu_data[tp] = data
                if i >= 4:  # Limite à 5 entrées (0 à 4)
                    break
            
            print(json.dumps(apercu_data, indent=2, ensure_ascii=False) + "\n...")
            print("-----------------------------------------------------")
            
        except json.JSONDecodeError:
            print(f"❌ ERREUR : Le LLM n'a pas renvoyé un JSON valide. Sortie brute : {thesaurus_json_str[:500]}...")
        except Exception as e:
             print(f"❌ ERREUR lors de la sauvegarde ou de l'affichage : {e}")

    print("\n--- Processus Terminé ---")

✅ Initialisation réussie (Gemini Client).
✅ 3509 tokens de concepts chargés pour le LLM.

[--- ENVOI DU PROMPT À GEMINI pour 20 TP ---]
Tentative 1/3...

✅ Thésaurus généré avec succès et enregistré dans 'Thesaurus_20_Concepts.json'.

--- APERÇU DU RÉSULTAT FINAL (Premières entrées) ---
{
  "Arrosage": {
    "BT": [
      "irrigation"
    ],
    "NT": [
      "bassinage",
      "aspersion",
      "drip"
    ],
    "RT": [
      "arroser",
      "eau",
      "humidit",
      "sol",
      "ollas"
    ],
    "UF": [
      "Arrosage des plantes",
      "Watering",
      "Irrigation des cultures",
      "Hydratation",
      "Apport en eau"
    ],
    "SN": "Action d'apporter de l'eau aux plantes pour satisfaire leurs besoins hydriques, cruciale pour leur survie et leur croissance."
  },
  "Luminosité": {
    "BT": [
      "lumire"
    ],
    "NT": [
      "photopriode"
    ],
    "RT": [
      "soleil",
      "exposition",
      "clairage",
      "photosynthse",
      "hliophile",
      "sc

In [ ]:
import os
import json
import time
from typing import List, Dict, Set
from google import genai
from google.genai.errors import APIError

# --- 1. CONFIGURATION GLOBALE ---

# Chemins
FICHIER_VOCABULAIRE_CONCEPTS = "token_concept.json" 
FICHIER_THESAURUS_SORTIE = "Thesaurus_20_Concepts_v3.json"

# Les 20 THÈMES directeurs. Le LLM doit SELECTIONNER un token d'index (TP) pour chacun.
THEMES_CONCEPTS: List[str] = [
    "Arrosage", "Luminosité", "Types de sol / substrat", "pH du sol", 
    "Température", "Humidité ambiante", "Engrais / nutrition NPK", "Rempotage", 
    "Plantation (mise en terre / pot)", "Drainage", "Propagation / reproduction", 
    "Taille et élagage", "Maladies fongiques", "Parasites courants", 
    "Toxicité (humains & animaux)", "Croissance et développement", 
    "Stress hydrique / thermique / lumineux", "Cycle de vie (annuelle / vivace / arbuste...)", 
    "Origine géographique & biomes", "Technologies horticoles modernes (LED, capteurs, hydroponie…)"
]

MAX_RETRIES = 5 # Augmenter le nombre de tentatives en cas d'erreur API 503 temporaire

# --- 2. INITIALISATION ET CHARGEMENT DU VOCABULAIRE ---

# 🚨 IMPORTANT : REMPLACEZ 'VOTRE_NOUVELLE_CLE_ICI' par la clé réelle.
# Attention : Ne pas partager cette version du script !
CLE_API_GEMINI = "here is API" 

try:
    # Initialisation explicite du client avec la nouvelle clé
    client = genai.Client(api_key=CLE_API_GEMINI) 
    print("✅ Initialisation réussie (Gemini Client avec clé explicite).")
except Exception as e:
    print(f"🚨 ERREUR d'initialisation du client : {e}")
    exit()

# ... le reste du script continue ...

def charger_vocabulaire_concepts(filepath: str) -> Set[str]:
    """Charge l'ensemble des tokens depuis le fichier JSON des concepts."""
    if not os.path.exists(filepath):
        print(f"🚨 ERREUR : Fichier vocabulaire non trouvé à {filepath}. Annulation.")
        return set()
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
            if isinstance(data, list):
                return set(data)
            else:
                print("🚨 ERREUR : Le fichier vocabulaire n'est pas une liste de mots.")
                return set()
    except Exception as e:
        print(f"❌ ERREUR lors du chargement du vocabulaire : {e}")
        return set()

# Chargement de tous les tokens du corpus concept
TERMES_CONCEPTS: Set[str] = charger_vocabulaire_concepts(FICHIER_VOCABULAIRE_CONCEPTS)

if not TERMES_CONCEPTS:
    print("\n❌ Arrêt : Vocabulaire de base vide ou introuvable.")
    exit()

print(f"✅ {len(TERMES_CONCEPTS)} tokens de concepts chargés pour le LLM.")


# --- 3. GÉNÉRATION PAR LLM ---

def generer_thesaurus_llm(themes_directeurs: List[str], vocabulaire: Set[str]) -> str:
    """
    Construit le prompt unique avec les tokens et appelle l'API Gemini pour générer les TP.
    """
    
    # 1. Sérialisation de la liste de vocabulaire
    vocabulaire_liste_str = json.dumps(sorted(list(vocabulaire)), ensure_ascii=False, indent=2)
    
    # 2. Construction du Prompt (Instructions Clés Mises à Jour)
    prompt_template = f"""
Role: Tu es un expert en conception de thésaurus (ISO 25964) pour la recherche botanique.
Objectif: Construire un thésaurus riche et structuré pour 20 thèmes directeurs.

Règles de Sélection des Termes Préférés (TP) (NOUVELLE RÈGLE):
1. Pour chacun des {len(themes_directeurs)} thèmes suivants : {themes_directeurs}, tu dois d'abord identifier le terme **le plus pertinent et le plus couramment utilisé** directement dans la "Liste de Vocabulaire Brut". Ce terme sera le **TP** (Terme Préféré) de l'entrée. Le TP DOIT être un token exact de la liste.
2. Tu dois générer **au moins 3 TP supplémentaires** pour chaque thème en sélectionnant d'autres termes pertinents dans le "Vocabulaire Brut" (mots scientifiques, techniques, synonymes) pour enrichir le thésaurus. Cela signifie environ **80 entrées TP au total** (20 principaux + 3 suppléments par thème).

Règles de Contenu (Similaires, mais renforcées pour l'enrichissement):
1. SÉLECTION EXCLUSIVE : Les valeurs pour les relations **NT, BT, et RT** DOIVENT ÊTRE CHOISIES **UNIQUEMENT** parmi les tokens exacts de la "Liste de Vocabulaire Brut" fournie.
2. ENRICHISSEMENT (UF) : La section **UF (Synonymes/Non-Descripteurs)** doit être enrichie avec des variations linguistiques (Darija, anglicismes, expressions communes). Ces termes sont les **SEULS** qui peuvent être **générés librement** par toi et ne pas être présents dans la "Liste de Vocabulaire Brut". Ajoute le nom du concept original dans l'UF pour le relier.
3. NOTE D'APPLICATION (SN) : Fournir une brève Note d'Application (SN) pour chaque TP.

Format de Sortie Strict:
Génère STRICTEMENT un seul objet JSON. Chaque clé de l'objet DOIT être un Terme Préféré (TP) choisi dans la liste de vocabulaire, et sa valeur doit être un objet contenant les champs BT, NT, RT, UF, et SN.

---
### DONNÉES D'ENTRÉE BRUTES DU CORPUS ({len(vocabulaire)} tokens)
---
Liste de Vocabulaire Brut (Tokens disponibles pour TP, NT, BT, RT):
{vocabulaire_liste_str}

--- SORTIE JSON ATTENDUE ---
"""
    
    print(f"\n[--- ENVOI DU PROMPT À GEMINI pour générer au moins {len(themes_directeurs) * 4} TP ---]")
    
    for attempt in range(MAX_RETRIES):
        try:
            print(f"Tentative {attempt + 1}/{MAX_RETRIES}...")
            response = client.models.generate_content(
                model='gemini-2.5-flash',
                contents=prompt_template,
                config={"response_mime_type": "application/json"}
            )
            return response.text
            
        except APIError as e:
            if '503 UNAVAILABLE' in str(e) and attempt < MAX_RETRIES - 1:
                delay = 15
                print(f"❌ Erreur 503 (Surchargé). Nouvelle tentative dans {delay} secondes...")
                time.sleep(delay)
            else:
                print(f"❌ ERREUR API critique : Échec après {attempt + 1} tentatives. Détail : {e}")
                return None
        except Exception as e:
            print(f"❌ ERREUR inattendue lors de l'appel LLM : {e}")
            return None
    
    return None 

# --- 4. SCRIPT PRINCIPAL (Exécution) ---

if __name__ == "__main__":
    
    # 1. GÉNÉRATION DU THESAURUS PAR LLM
    thesaurus_json_str = generer_thesaurus_llm(THEMES_CONCEPTS, TERMES_CONCEPTS)
    
    # 2. SAUVEGARDE ET AFFICHAGE
    if thesaurus_json_str:
        try:
            thesaurus_data = json.loads(thesaurus_json_str)
            
            with open(FICHIER_THESAURUS_SORTIE, 'w', encoding='utf-8') as f:
                json.dump(thesaurus_data, f, ensure_ascii=False, indent=2)
                
            print(f"\n✅ Thésaurus généré avec succès et enregistré dans '{FICHIER_THESAURUS_SORTIE}'.")
            print(f"Le fichier contient {len(thesaurus_data)} Termes Préférés (TP) basés sur votre vocabulaire d'index.")
            
            # Affichage d'un aperçu
            print("\n--- APERÇU DU RÉSULTAT FINAL (Exemple d'une entrée) ---")
            
            # Afficher la première entrée pour voir la nouvelle structure
            premier_tp = next(iter(thesaurus_data))
            
            print(json.dumps({premier_tp: thesaurus_data[premier_tp]}, indent=2, ensure_ascii=False))
            print("-----------------------------------------------------")
            
        except json.JSONDecodeError:
            print(f"❌ ERREUR : Le LLM n'a pas renvoyé un JSON valide. Sortie brute : {thesaurus_json_str[:500]}...")
        except Exception as e:
             print(f"❌ ERREUR lors de la sauvegarde ou de l'affichage : {e}")

    print("\n--- Processus Terminé ---")

✅ Initialisation réussie (Gemini Client avec clé explicite).
✅ 3592 tokens de concepts chargés pour le LLM.

[--- ENVOI DU PROMPT À GEMINI pour générer au moins 80 TP ---]
Tentative 1/5...
❌ Erreur 503 (Surchargé). Nouvelle tentative dans 15 secondes...
Tentative 2/5...
❌ Erreur 503 (Surchargé). Nouvelle tentative dans 15 secondes...
Tentative 3/5...
❌ Erreur 503 (Surchargé). Nouvelle tentative dans 15 secondes...
Tentative 4/5...
❌ Erreur 503 (Surchargé). Nouvelle tentative dans 15 secondes...
Tentative 5/5...
❌ ERREUR API critique : Échec après 5 tentatives. Détail : 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}

--- Processus Terminé ---
